In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
df = pd.read_csv('OnlineNewsPopularity.csv')

In [3]:
def first_n_words(text,sequence_length = 100):
    # Split the text into individual words
    words = text.split()
    text = ''
    # Print the first 100 words
    for i in range(sequence_length):
        if i < len(words):
            text += words[i] + ' '
        else:
            break
    return text
# Send a GET request to the URL
def get_news(url,sequence_length = 100):
    headline_text = ''
    body_text = ''
    published_date_text = ''
    response = requests.get(url)
    # Create a BeautifulSoup object to parse the HTML content
    soup = BeautifulSoup(response.content, "html.parser")    
    # Find the elements containing the news headlines/body/published_date  
    
    headlines = soup.find_all("h1")
    bodys = soup.find_all("p")
    times = soup.find_all("time")
    
    # Extract and print the headlines
    for headline in headlines:
        headline_text += headline.text
    for body in bodys:
        body_text += body.text
    for time in times:
        published_date_text += time.text       
    return(headline_text,first_n_words(body_text,sequence_length),published_date_text)

In [4]:
n = 100
urls = df['url'][:n]
news_headline = []
news_body = []
news_published_date = []
counter = 1
for url in urls:
    news_headline.append(get_news(url,100)[0])
    news_body.append(get_news(url,100)[1])
    news_published_date.append(get_news(url,100)[2])
    counter += 1
    if counter%100 == 0:
        print(counter)

100


In [5]:
news_df = pd.DataFrame(list(zip(urls,
                               news_headline,
                               news_body,
                               news_published_date,
                               df[' global_sentiment_polarity'][:n],
                               df[' shares'][:n])))
news_df.columns = ['url',
                   'headline',
                   'body',
                  'publish_date',
                  'global_sentiment_polarity',
                  'shares']

news_df['publish_date'] = pd.to_datetime(news_df['publish_date'])


In [6]:
news_df.to_csv('news_popularity.csv',index=False
            )